# Pandas notes

In [ ]:
from IPython.display import Image, SVG

Image(filename='Images/multtimeseries.png')
SVG(filename='Images/multtimeseriesslice1.svg')

# 1. Create untidy data with - pd.melt()
- melt = turn columns into rows

In [ ]:
"""
params:
id_vars - columns to keep fixed, NOT melt
value_vars - columns to melt, default to melt all columns
var_name and value_name - rename columns
"""
pd.melt(frame=df, id_vars='name', value_vars=['treatment a', 'treatment b'],
       var_name='treatment', value_name='result')

In [ ]:
# Example: melt tidy data into untidy form

# Combine Ozone, Solar.R, Wind, and Temp columns into 1 column

# Print the head of airquality
print(airquality.head())
#    Ozone  Solar.R  Wind  Temp  Month  Day
# 0   41.0    190.0   7.4    67      5    1
# 1   36.0    118.0   8.0    72      5    2
# 2   12.0    149.0  12.6    74      5    3
# 3   18.0    313.0  11.5    62      5    4
# 4    NaN      NaN  14.3    56      5    5

# Melt airquality: airquality_melt
airquality_melt = pd.melt(frame=airquality, id_vars=['Month', 'Day'])

# Print the head of airquality_melt
print(airquality_melt.head())
#    Month  Day variable  value
# 0      5    1    Ozone   41.0
# 1      5    2    Ozone   36.0
# 2      5    3    Ozone   12.0
# 3      5    4    Ozone   18.0
# 4      5    5    Ozone    NaN

In [ ]:
# specify melted column names with var_name and value_name arg
# Melt airquality: airquality_melt
airquality_melt = pd.melt(frame=airquality, id_vars=['Month', 'Day'], 
                          var_name='measurement', value_name='reading')

# 2. Pivot - un-melt data with pivot()
- opposite of melting
- pivot = turn unique values into separate columns
- args:
    - index - columns you don't want to pivot
    - columns - columns you want to pivot
    - values - values to use

In [ ]:
weather_tidy = weather.pivot(index='date', columns='element', values='value')

## 2.1 Using pivot_table

In [ ]:
# Print the head of airquality_melt
print(airquality_melt.head())
#        Month  Day measurement  reading
#     0      5    1       Ozone     41.0
#     1      5    2       Ozone     36.0
#     2      5    3       Ozone     12.0
#     3      5    4       Ozone     18.0
#     4      5    5       Ozone      NaN
# Pivot airquality_melt: airquality_pivot
airquality_pivot = airquality_melt.pivot_table(index=['Month', 'Day'], 
                                               columns='measurement', 
                                               values='reading')

# Print the head of airquality_pivot
print(airquality_pivot.head())
#     measurement  Ozone  Solar.R  Temp  Wind
#     Month Day                              
#     5     1       41.0    190.0  67.0   7.4
#           2       36.0    118.0  72.0   8.0
#           3       12.0    149.0  74.0  12.6
#           4       18.0    313.0  62.0  11.5
#           5        NaN      NaN  56.0  14.3

## 2.2 Resetting index of a dataframe (get rid of multiindex)

In [ ]:
# Print the index of airquality_pivot
print(airquality_pivot.index)

# Reset the index of airquality_pivot: airquality_pivot_reset
airquality_pivot_reset = airquality_pivot.reset_index()

# Print the new index of airquality_pivot_reset
print(airquality_pivot_reset.index)

# Print the head of airquality_pivot_reset
print(airquality_pivot_reset.head())

## 2.3 Using pivot when you have duplicate entries - pivot_table()
- has parameter to deal with duplicate values
- example: can aggregate the duplicate values by taking their average

In [ ]:
# pivot_table with aggfunc
weather2_tidy = weather.pivot_table(values = 'value',
                                    index='date', 
                                    columns='element', 
                                    aggfunc=np.mean)

In [ ]:
# Pivot airquality_dup: airquality_pivot
airquality_pivot = airquality_dup.pivot_table(index=['Month', 'Day'], 
                                              columns='measurement', 
                                              values='reading', 
                                              aggfunc=np.mean)

# Reset the index of airquality_pivot
airquality_pivot = airquality_pivot.reset_index()

# Print the head of airquality_pivot
print(airquality_pivot.head())

# Print the head of airquality
print(airquality.head())


# 3. Melting and Parsing
- separate 1 column with value that has 2 elements - into 2 separate columns

In [ ]:
# variable column has age and sex - ie. m014
pd.melt(frame=tb, id_vars=['country', 'year'])
# create 'gender' column
tb_melt['gender'] = tb_melt.variable.str[0]
# create age column
tb_melt['age_group'] = tb_melt.variable.str[1:]

# 4. Splitting a column with .split() and .get()
- use when multiple variables are stored in columns with a delimiter, ie. '_'
    - ie. column name Deaths_Guinea

In [ ]:
# Melt ebola: ebola_melt
ebola_melt = pd.melt(ebola, id_vars=['Date', 'Day'], var_name='type_country', value_name='counts')

# Create the 'str_split' column
ebola_melt['str_split'] = ebola_melt.type_country.str.split('_')

# Create the 'type' column
ebola_melt['type'] = ebola_melt['str_split'].str.get(0)

# Create the 'country' column
ebola_melt['country'] = ebola_melt['str_split'].str.get(1)

# Print the head of ebola_melt
print(ebola_melt.head())


# 5. Concatenating many files - glob() to find files based on pattern
Globbing
- pattern matching for file names
- Wildcards: *?
    - '*' for any string
        - example: Any csv file: *.csv
    - ? for one character
        - example: Any single character: file_?.csv
- Returns a list of file names
- Use this list to load into separate DataFrames

Plan
- load files from globbing into pandas
- add dataframes into a list
- concatenate multiple datasets at once

In [ ]:
import glob
# find and concatenate
csv_files = glob.glob('*.csv')
print(csv_files)

# create list of dataframes using a for loop 
list_data = []
for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)
pd.concat(list_data)

In [ ]:
# example: Find files that match a pattern

# Import necessary modules
import glob
import pandas as pd

# Write the pattern: pattern
pattern = '*.csv'

# Save all file matches: csv_files
csv_files = glob.glob(pattern)

# Print the file names
print(csv_files)

# Load the second file into a DataFrame: csv2
csv2 = pd.read_csv(csv_files[1])

# Print the head of csv2
print(csv2.head())

In [ ]:
# example: iterate and concatenate df for all matches
# Create an empty list: frames
frames = []

#  Iterate over csv_files
for csv in csv_files:

    #  Read csv into a DataFrame: df
    df = pd.read_csv(csv)
    
    # Append df to frames
    frames.append(df)

# Concatenate frames into a single DataFrame: uber
uber = pd.concat(frames)

# Print the shape of uber
print(uber.shape)

# Print the head of uber
print(uber.head())

# 6. Merge
3 types
- one to one
- many to one / one to many
- many to many

Use same function pd.merge()

## 6.1 merge: one to one

In [ ]:
pd.merge(left=state_populations, right=state_codes,
        on=None, left_on='state', right_on='name')

In [ ]:
# Merge the DataFrames: o2o
o2o = pd.merge(left=site, right=visited, left_on='name', right_on='site')

## 6.2 merge: many to one / one to many
- for duplicates in key, both DataFrames do not have unique keys for a merge 
- What happens here is that for each duplicated key, every pairwise combination will be created.

In [ ]:
# Merge the DataFrames: m2o
m2o = pd.merge(left=site, right=visited, left_on='name', right_on='site')

## 6.3 merge: many to many

In [ ]:
# Merge site and visited: m2m
m2m = pd.merge(left=site, right=visited, left_on='name', right_on='site')

# Merge m2m and survey: m2m
m2m = pd.merge(left=m2m, right=survey, left_on='ident', right_on='taken')

# 7. Data types

In [ ]:
df.dtypes
df['column'].astype(str)
df['column'].astype(categorical)

In [ ]:
# object (string) to numeric type with coerce (coerces NaNs)
pd.to_numeric(df['column a'], errors='coerce')

# 8. String manipulation with regex
- import re

Examples
- 17   \d*
- $17  \$\d*
- $17.00 \$\d*\.\d*
- Specify 2 digit decimal: $17.89
    - \$\d*\.\d{2}
- ^ at beginning and $ at end
- $17.895  ^\$\d*\.\d{2}$

## 8.1 Using regular expressions (aka regex)
- compile the pattern
- use the pattern to match values


In [ ]:
# Example: regex
import re
pattern = re.compile('\$\d*\.\d{2}')
result = pattern.match('$17.89')
bool(result)

In [ ]:
# Example: regex for phone number matching
# Import the regular expression module
import re

# Compile the pattern: prog
prog = re.compile('\d{3}-\d{3}-\d{4}')

# See if the pattern matches
result = prog.match('123-456-7890')
print(bool(result))

# See if the pattern matches
result2 = prog.match('1123-456-7890')
print(bool(result2))

# output
# True
# False

In [ ]:
# Example: extract numbers from strings
# Find numbers in string: 'the recipe calls for 10 strawberries and 1 banana'
# Import the regular expression module
import re

# Find the numeric values: matches
matches = re.findall('\d+', 'the recipe calls for 10 strawberries and 1 banana')

# Print the matches
print(matches)

In [ ]:
# Examples: mix of regex patterns
# Write the first pattern
pattern1 = bool(re.match(pattern='\d{3}-\d{3}-\d{4}', string='123-456-7890'))
print(pattern1)

# Write the second pattern
pattern2 = bool(re.match(pattern='\$\d{3}', string='$123.45'))
print(pattern2)

# Write the third pattern
pattern3 = bool(re.match(pattern='\d*', string='Australia'))
print(pattern3)

## 8.2 Custom functions to clean data using regex

In [ ]:
# Example: use a custom function to apply regex to create a new column
import re
from numpy import NaN
pattern = re.compile('^\$\d*\.\d{2}$')

# function
def diff_money(row, pattern):
    # slice the row
    icost = row['Initial Cost']
    tef = row['Total Est. Fee']
    
    # if valid value, then convert string to float type
    if bool(pattern.match(icost)) and bool(pattern.match(tef)):
        icost = icost.replace("$", "")
        tef = tef.replace("$","")
        
        icost = float(icost)
        tef = float(tef)
        
        # return difference of the values
        return icost - tef
    else:
        return(NaN)
    
# apply function
df_subset['diff'] = df_subset.apply(diff_money, axis=1, pattern=pattern)

### 8.2.1 Custom functions to clean data
You'll now practice writing functions to clean data.

The tips dataset has been pre-loaded into a DataFrame called tips. It has a 'sex' column that contains the values 'Male' or 'Female'. Your job is to write a function that will recode 'Male' to 1, 'Female' to 0, and return np.nan for all entries of 'sex' that are neither 'Male' nor 'Female'.

Recoding variables like this is a common data cleaning task. Functions provide a mechanism for you to abstract away complex bits of code as well as reuse code. This makes your code more readable and less error prone.

As Dan showed you in the videos, you can use the .apply() method to apply a function across entire rows or columns of DataFrames. However, note that each column of a DataFrame is a pandas Series. Functions can also be applied across Series. Here, you will apply your function over the 'sex' column.

In [ ]:
# Define recode_sex()
def recode_sex(sex_value):

    # Return 1 if sex_value is 'Male'
    if sex_value == 'Male':
        return 1
    
    # Return 0 if sex_value is 'Female'    
    elif sex_value == 'Female':
        return 0
    
    # Return np.nan    
    else:
        return np.nan

# Apply the function to the sex column
tips['sex_recode'] = tips.sex.apply(recode_sex)

# Print the first five rows of tips
print(tips.head())

### 8.2.2 Lambda functions

In [ ]:
# Example: try removing $ sign from a column using 2 methods
# Method 1: use .replace(), Method 2: use regex

# Write the lambda function using replace
tips['total_dollar_replace'] = tips.total_dollar.apply(
    lambda x: x.replace('$', ''))

# Write the lambda function using regular expressions
tips['total_dollar_re'] = tips.total_dollar.apply(
    lambda x: re.findall('\d+\.\d+', x)[0])

# Print the head of tips
print(tips.head())


# 9. Duplicate data
- .drop_duplicates()

In [ ]:
# drop duplicates
df = df.drop_duplicates()

# 10. Missing data
Handling options:
- leave as is
- drop them
- fill missing value

Question
- is it random
- or is it a systemic problem

In [ ]:
# Drop missing values
tips_dropped = tips_nan.dropna()
tips_dropped info()

In [ ]:
# Fill missing values with .fillna() - options: provided value, summary statistic
tips_nan['sex'] = tips_nan['sex'].fillna('missing')
# fill mssing values from multiple columns
tips_nan[['total_bill', 'size']] = tips_nan[['total_bill', 'size']].fillna(0)

In [ ]:
# Fill missing value with a test statistic
mean_value = tips_nan['tip'].mean()
print(mean_value)
tips_nan['tip'] = tips_nan['tip'].fillna(mean_value)
tips_nan.info()

In [ ]:
# Drop duplicates
# Create the new DataFrame: tracks
tracks = billboard[['year', 'artist', 'track', 'time']]

# Drop the duplicates: tracks_no_duplicates
tracks_no_duplicates = tracks.drop_duplicates()

# 11. Testing with Assert statements
Assert statements
- programmatically vs visually checking for NaNs
- if we drop of fill NaNs, we expect 0 missing values
- We can write an assert statement to verify this
- we can detect early warnings and errors

In [ ]:
# Assert statment functionality

# if true, return nothing
assert 1 == 1

# if false, give Error
assert 1 == 2
# AssertionError ...

In [ ]:
# Example: Test column for null
assert df.column.notnull().all()
# Assertion error since missing values exist

# Fill missing values with 0
df_0 = df.fillna(value=0)

In [ ]:
# Example: Test entire dataframe for (1) null and (2) values >= 0
# Assert that there are no missing values
assert ebola.notnull().all().all()

# Assert that all values are >= 0
assert (ebola >= 0).all().all()

# 12. Case example: load, EDA, check data quality

In [ ]:
import pandas as pd
df = pd.read_csv('my_data.csv')
df.head()
df.info()
df.columns
df.describe()
df.shape
df.column.value_counts()
df.column.plot('hist')

# function to check for data quality issues
def cleaning_function(row_data):
    # data cleaning steps
    return ...

# apply function to row
df.apply(cleaning_function, axis=1)

# assert statement
assert (df.column_data > 0).all()

In [ ]:
# Example of verifying data quality

# Function to verify assumptions of data
def check_null_or_valid(row_data):
    """Function that takes a row of data,
    drops all missing values,
    and checks if all remaining values are greater than or equal to 0
    """
    # start at 1 since column 0 is a string type
    no_na = row_data.dropna()[1:-1]
    numeric = pd.to_numeric(no_na)
    ge0 = numeric >= 0
    return ge0

# Check whether the first column is 'Life expectancy'
assert g1800s.columns[0] == 'Life expectancy'

# Check whether the values in the row are valid
assert g1800s.iloc[:, 1:].apply(check_null_or_valid, axis=1).all().all()

# Check that there is only one instance of each country
assert g1800s['Life expectancy'].value_counts()[0] == 1

In [ ]:
# Tidy data with melt (turn columns into rows) and 
# pivot (take unique values from a column and create new columns)

# Melt gapminder: gapminder_melt
gapminder_melt = pd.melt(frame=gapminder, id_vars='Life expectancy')

# Rename the columns
gapminder_melt.columns = ['country','year','life_expectancy']

# Print the head of gapminder_melt
print(gapminder_melt.head())

In [ ]:
# Check dtypes
# Convert the year column to numeric
gapminder.year = pd.to_numeric(gapminder.year)

# Test if country is of type object
assert gapminder.country.dtypes == np.object

# Test if year is of type int64
assert gapminder.year.dtypes == np.int64

# Test if life_expectancy is of type float64
assert gapminder.life_expectancy.dtypes == np.float64

In [ ]:
# Check 'country' column for spellings
# Create the series of countries: countries
countries = gapminder.country

# Drop all the duplicates from countries
countries = countries.drop_duplicates()

# Write the regular expression: pattern
pattern = '^[A-Za-z\.\sA-Za-z]*$'

# Create the Boolean vector: mask
mask = countries.str.contains(pattern)

# Invert the mask: mask_inverse
mask_inverse = ~mask

# Subset countries using mask_inverse: invalid_countries
invalid_countries = countries.loc[mask_inverse]

# Print invalid_countries
print(invalid_countries)

In [ ]:
# Deal with missing data
# Assert that country does not contain any missing values
assert pd.notnull(gapminder.country).all()

# Assert that year does not contain any missing values
assert pd.notnull(gapminder.year).all()

# Drop the missing values - should only affect 'country' column due to asserts
gapminder = gapminder.dropna()

# Print the shape of gapminder
print(gapminder.shape)

In [ ]:
# Data visualization and aggregation
## Add first subplot
plt.subplot(2, 1, 1) 

# Create a histogram of life_expectancy
gapminder.life_expectancy.plot(kind='hist')

# Group gapminder: gapminder_agg
gapminder_agg = gapminder.groupby('year')['life_expectancy'].mean()

# Print the head of gapminder_agg
print(gapminder_agg.head())

# Print the tail of gapminder_agg
print(gapminder_agg.tail())

# Add second subplot
plt.subplot(2, 1, 2)

# Create a line plot of life expectancy per year
gapminder_agg.plot()

# Add title and specify axis labels
plt.title('Life expectancy over the years')
plt.ylabel('Life expectancy')
plt.xlabel('Year')

# Display the plots
plt.tight_layout()
plt.show()

# Save both DataFrames to csv files
gapminder.to_csv('gapminder.csv')
gapminder_agg.to_csv('gapminder_agg.csv')

In [ ]:
Image(filename='Images/multtimeseries.png')
SVG(filename='Images/multtimeseriesslice1.svg')